![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [36]:
import pandas as pd
import numpy as np

# Start coding here...
bank_marketing = pd.read_csv("bank_marketing.csv")
#print(bank_marketing.head())

client = bank_marketing[["client_id","age","job","marital","education","credit_default","mortgage"]]

client["job"] = client["job"].str.replace(".","_")
client["education"] = client["education"].str.replace(".","_")
client["education"].loc[client["education"] == "unknown"] = np.nan
client["credit_default"] = client["credit_default"].str.replace("unknown","0")
client["credit_default"] = client["credit_default"].str.replace("no","0")
client["credit_default"] = client["credit_default"].str.replace("yes","1")
client["mortgage"] = client["mortgage"].str.replace("no","0")
client["mortgage"] = client["mortgage"].str.replace("yes","1")
#client["mortgage"].loc[client["mortgage"]=="unk0wn"] = np.nan
client["mortgage"] = client["mortgage"].str.replace("unk0wn","0")
client["mortgage"] = client["mortgage"].astype("int").astype(bool)
client["credit_default"] = client["credit_default"].astype("int").astype(bool)
#print(client['credit_default'].unique())
print(client.head())
client.to_csv('client.csv',index=False)

campaign = bank_marketing[['client_id','number_contacts','contact_duration',
                          'previous_campaign_contacts','previous_outcome','campaign_outcome']]
#print(campaign.head())
#print(campaign.info())
#print(campaign['previous_outcome'].unique())
#print(campaign['campaign_outcome'].unique())
campaign['previous_outcome']=campaign['previous_outcome'].str.replace("nonexistent","0")
campaign['previous_outcome']=campaign['previous_outcome'].str.replace("failure","0")
campaign['previous_outcome']=campaign['previous_outcome'].str.replace("success","1")
#print(campaign['previous_outcome'].unique())
campaign['previous_outcome']=campaign['previous_outcome'].astype("int").astype(bool)
campaign['campaign_outcome']=campaign['campaign_outcome'].str.replace("no","0")
campaign['campaign_outcome']=campaign['campaign_outcome'].str.replace("yes","1")
#print(campaign['campaign_outcome'].unique())
campaign['campaign_outcome']=campaign['campaign_outcome'].astype("int").astype(bool)
#print(campaign['previous_outcome'].unique())
#print(campaign['campaign_outcome'].unique())
campaign['year'] = '2022'
campaign['month'] = bank_marketing['month'].str.capitalize()
campaign['day'] = bank_marketing['day'].astype('str')
campaign['last_contact_date'] = campaign['year'] + "-" + campaign['month'] + "-" + campaign['day']
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'])
campaign.drop('month',axis='columns',inplace = True)
campaign.drop('day',axis='columns',inplace = True)
campaign.drop('year',axis='columns',inplace = True)
print(campaign.head())
campaign.to_csv('campaign.csv',index = False)

economics = bank_marketing[['client_id','cons_price_idx','euribor_three_months']]
print(economics.head())
economics.to_csv('economics.csv',index = False)

   client_id  age        job  marital    education  credit_default  mortgage
0          0   56  housemaid  married     basic_4y           False     False
1          1   57   services  married  high_school           False     False
2          2   37   services  married  high_school           False      True
3          3   40     admin_  married     basic_6y           False     False
4          4   56   services  married  high_school           False     False
   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1  ...             False         2022-05-23
3          3                1  ...             False         2022-05-27
4          4                1  ...             False         2022-05-03

[5 rows x 7 columns]
   client_id  cons_price_idx  euribor_three_months
0          0          93.994     